In [0]:
storage_account_name = "amazonelectron1446226083"
storage_account_key = "U4+rIh/mSWDtJBdiwl0UfTjOn7tEtlRxCgWIef58DxOwglDGAaImibpXUsiP7psHoE6yLZjlqFyG+ASt8JyBzA=="
spark.conf.set(
 f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
 storage_account_key
)


In [0]:
reviews_path = "abfss://processed@amazonelectron1446226083.dfs.core.windows.net/reviews/"
reviews_df = spark.read.parquet(reviews_path)

In [0]:
enriched_reviews_path = "abfss://processed@amazonelectron1446226083.dfs.core.windows.net/enriched_reviews/"
enriched_df = spark.read.parquet(enriched_reviews_path)


In [0]:
features_v1_df = enriched_df.select(
 "asin",
 "title",
 "brand",
 "price",
 "reviewerID",
 "overall",
 "summary",
 "reviewText",
 "helpful",
 "reviewTime",
 "review_year"
)

In [0]:
# Create the curated container if it doesn't exist
try:
    dbutils.fs.ls("abfss://curated@amazonelectron1446226083.dfs.core.windows.net/")
    print("Container 'curated' already exists")
except Exception as e:
    if "does not exist" in str(e).lower():
        print("Container 'curated' needs to be created in Azure Portal")
        print("Go to your storage account and create a container named 'curated'")
    else:
        raise

In [0]:
gold_path = "abfss://processed@amazonelectron1446226083.dfs.core.windows.net/features_v1/"
features_v1_df.write.mode("overwrite").parquet(gold_path)

In [0]:
gold_df = spark.read.parquet(gold_path)
gold_df.printSchema()
display(gold_df.limit(5))
print("Final row count:", gold_df.count())
